## imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import os
from os.path import join, exists
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.autograd import Variable as V
from torch.nn import functional as F
import torch.nn  as nn
from PIL import Image
from IPython.core.debugger import set_trace
import nethook as nethook
import scipy.io as sio
from tqdm import tqdm
import time

## hyperparameters

In [7]:
arch = 'AlexNet'
imageset = 'MC8-LumMatched'
imgtype = 'experimental'

## directory management

In [9]:
home_dir = '/home/jacobpri/git/RotspaceIT/'

# target directory for saving activations
activation_savedir = join(home_dir,'data','d02_modeling','activations')

# folder of trained model weights
weight_dir = join(home_dir,'data','d02_modeling','weights')

# folder of image sets
imageset_dir = join(home_dir,'imagesets',imgtype)

# create savedir if it doesn't yet exist 
savedir = os.path.join(activation_savedir, arch, imageset)
if not os.path.exists(savedir): os.makedirs(savedir)
print("save destination: %s" % savedir)

assert(exists(imageset_dir))
assert(exists(weight_dir))

save destination: /home/jacobpri/git/RotspaceIT/data/d02_modeling/activations/AlexNet/MC8-LumMatched


## load architecture

## convert to instrumented model

## retain all layers